In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import udf
from pyspark.ml.linalg import SparseVector, Vectors, VectorUDT
from pyspark.sql.types import ArrayType, IntegerType

In [2]:
spark = SparkSession \
    .builder \
    .appName("Keyword clustering") \
    .getOrCreate()

In [3]:
spark.version

'3.1.1'

In [4]:
sc = spark.sparkContext

In [5]:
df = spark.read.csv('./search_results.csv', header=True)

In [6]:
df = df.drop('Title', '_id')

In [7]:
df.show(3)

+--------------------+--------------------+
|             Keyword|                 Url|
+--------------------+--------------------+
|sump pump next to...|https://www.mlive...|
|sump pump next to...|https://www.famil...|
|sump pump next to...|https://diy.stack...|
+--------------------+--------------------+
only showing top 3 rows



In [8]:
df.columns

['Keyword', 'Url']

In [9]:
keyword2url = df.groupBy('Keyword') \
  .agg(F.collect_list('url').alias('urlIndex'))

In [10]:
keyword2url.show()

+--------------------+--------------------+
|             Keyword|            urlIndex|
+--------------------+--------------------+
|1/4 vs 1/3 sump pump|[https://www.ag.n...|
|2 stage dry sump ...|[https://www.drys...|
|3 phase sump pump...|[https://www.sept...|
|6-cia-ml 1/3 hp s...|[https://www.home...|
|add battery to ex...|[https://combatwa...|
|add battery to su...|[https://resource...|
|adjusting zoeller...|[https://www.zoel...|
|average sump pump...|[https://www.rado...|
|battery backup su...|[https://www.home...|
|best sump pump fo...|[https://www.pres...|
|best way to drain...|[https://www.base...|
|big john automati...|[https://www.norr...|
|black friday sump...|[https://www.amaz...|
|bur cam 1/3 hp su...|[https://www.sump...|
|campbell hausfeld...|[https://www.mena...|
|can you connect s...|[https://www.roto...|
|changing out sump...|[https://www.fres...|
|clogged sump pump...|[https://homeguid...|
|coleman effluent ...|[https://www.amaz...|
|consumer report b...|[https://b

In [11]:
df3 = keyword2url.withColumn("k_id", F.monotonically_increasing_id())
df3.createOrReplaceTempView('df3')
df3 = spark.sql('select row_number() over (order by "k_id") as Keyword_id, * from df3').drop('k_id')

In [12]:
df3.show(135)

+----------+--------------------+--------------------+
|Keyword_id|             Keyword|            urlIndex|
+----------+--------------------+--------------------+
|         1|1/4 vs 1/3 sump pump|[https://www.ag.n...|
|         2|2 stage dry sump ...|[https://www.drys...|
|         3|3 phase sump pump...|[https://www.sept...|
|         4|6-cia-ml 1/3 hp s...|[https://www.home...|
|         5|add battery to ex...|[https://combatwa...|
|         6|add battery to su...|[https://resource...|
|         7|adjusting zoeller...|[https://www.zoel...|
|         8|average sump pump...|[https://www.rado...|
|         9|battery backup su...|[https://www.home...|
|        10|best sump pump fo...|[https://www.pres...|
|        11|best way to drain...|[https://www.base...|
|        12|big john automati...|[https://www.norr...|
|        13|black friday sump...|[https://www.amaz...|
|        14|bur cam 1/3 hp su...|[https://www.sump...|
|        15|campbell hausfeld...|[https://www.mena...|
|        1

In [20]:
df3.where(df3.Keyword == "my sump pump smells like sewage").select("Keyword_id").show()

+----------+
|Keyword_id|
+----------+
|     14188|
+----------+



In [13]:
df4 = df3.withColumn("Url", F.explode(df3.urlIndex)) \
      .drop("urlIndex") \
      #.withColumn("url_id", F.monotonically_increasing_id())

In [14]:
df4.show(truncate = 40)

+----------+---------------------+----------------------------------------+
|Keyword_id|              Keyword|                                     Url|
+----------+---------------------+----------------------------------------+
|         1| 1/4 vs 1/3 sump pump|https://www.ag.ndsu.edu/publications/...|
|         1| 1/4 vs 1/3 sump pump|https://www.ag.ndsu.edu/publications/...|
|         1| 1/4 vs 1/3 sump pump|https://www.sumppumpsdirect.com/stori...|
|         1| 1/4 vs 1/3 sump pump|https://www.sumppumpsdirect.com/stori...|
|         1| 1/4 vs 1/3 sump pump|https://www.bluefrosthvac.com/blog/ch...|
|         1| 1/4 vs 1/3 sump pump|https://www.1tomplumber.com/everythin...|
|         1| 1/4 vs 1/3 sump pump|https://www.bobvila.com/articles/best...|
|         1| 1/4 vs 1/3 sump pump|https://www.homeownershub.com/mainten...|
|         1| 1/4 vs 1/3 sump pump|https://solvitnow.com/best-sump-pumps...|
|         1| 1/4 vs 1/3 sump pump|https://www.homeconstructionimproveme...|
|         1|

In [19]:
df4.where(df4.Keyword == "my sump pump smells like sewage").select("Keyword_id").show()

+----------+
|Keyword_id|
+----------+
|     14208|
|     14208|
|     14208|
|     14208|
|     14208|
|     14208|
|     14208|
|     14208|
|     14208|
|     14208|
+----------+



In [18]:
df4.where(df4.Keyword == "my sump pump smells like sewage").select("Url").show()

+--------------------+
|                 Url|
+--------------------+
|https://www.callk...|
|https://mrplumber...|
|https://apollohom...|
|https://www.homea...|
|https://floodnot....|
|https://www.preci...|
|https://www.ameri...|
|https://www.sumpp...|
|http://www.capita...|
|https://www.remod...|
+--------------------+



In [108]:
keyword_ids = df3.select('Keyword_id', 'Keyword')
keyword_ids.coalesce(1).write.csv('keywords.csv')

In [95]:
url2keyword = df4.drop('Keyword').groupBy('Url') \
    .agg(F.collect_list('Keyword_id').alias('Keywords_ids'))

In [96]:
url2keyword = url2keyword.withColumn("u_id", F.monotonically_increasing_id())
url2keyword.createOrReplaceTempView('url2keyword')
url2keyword = spark.sql('select row_number() over (order by "u_id") as Url_id, * from url2keyword').drop('u_id')

In [97]:
url2keyword.show()

+------+--------------------+--------------------+
|Url_id|                 Url|        Keywords_ids|
+------+--------------------+--------------------+
|     1|        (5 Year ..."|              [9948]|
|     2|              1 ..."|             [18139]|
|     3|                1 HP|             [16314]|
|     4|          1-1/2 ..."|[7514, 10802, 109...|
|     5|    1-1/2"" - - ..."|             [18967]|
|     6| 1-1/2"" - - Amaz...|[508, 605, 761, 8...|
|     7| 1-1/2"" - Amazon...|      [18068, 20569]|
|     8|   1-1/2"" - Amazon"|       [4701, 17570]|
|     9| 1-1/2"" - Amazon...|       [6822, 11810]|
|    10| 1-1/2"" - Amazon...|             [18068]|
|    11| 1-1/2"" - Amazon...|[7854, 14149, 23288]|
|    12|   1-1/2""- Buy ..."|             [18068]|
|    13|            1/2 ..."|             [24326]|
|    14| 1/3 HP Stainless...|             [23890]|
|    15|            1/4 ..."|      [13208, 24943]|
|    16|             10 ..."|             [20142]|
|    17|         10 GPM ..."|  

In [107]:
url_ids = url2keyword.select('Url_id', 'Url')
url_ids.coalesce(1).write.csv('urls.csv')

In [100]:
df5 = url2keyword.withColumn("Keyword_id", F.explode(url2keyword.Keywords_ids)) \
      .drop("Keywords_ids", "Url")

In [101]:
df5.show()

+------+----------+
|Url_id|Keyword_id|
+------+----------+
|     1|      9948|
|     2|     18139|
|     3|     16314|
|     4|      7514|
|     4|     10802|
|     4|     10986|
|     4|     12846|
|     4|     13512|
|     4|     14963|
|     4|     14992|
|     4|     18068|
|     4|     18068|
|     4|     18123|
|     4|     19200|
|     4|     21437|
|     4|     21842|
|     4|     22287|
|     4|     23288|
|     4|     24268|
|     4|     26403|
+------+----------+
only showing top 20 rows



In [106]:
df5.coalesce(1).write.csv('url2keyword.csv')